# Report Generation for Heart Failure Prediction
Here's an example to demonstrate how we can generate a report as we proceed through all the steps to train and evaluate a model. For this purpose, we are going to use Kaggle's heart prediction failure dataset and gradually populate the report with information about dataset, model and results.

In [ ]:
"""Imports."""

import copy
import inspect
import os
import shutil
from datetime import date

import numpy as np
import pandas as pd
import plotly.express as px
from datasets import Dataset
from kaggle.api.kaggle_api_extended import KaggleApi
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

from cyclops.data.slicer import SliceSpec
from cyclops.evaluate import evaluator
from cyclops.evaluate.metrics import create_metric
from cyclops.evaluate.metrics.experimental.metric_dict import MetricDict
from cyclops.report import ModelCardReport
from cyclops.report.plot.classification import ClassificationPlotter
from cyclops.report.utils import flatten_results_dict

## Create Model Card Report
First, we should create a `ModelCardReport` object to fill in the fields and sections after training.

In [ ]:
report = ModelCardReport()

In [ ]:
# Constants
DATA_DIR = "./data"
RANDOM_SEED = 21

## Data Loading
Before starting, make sure to install the Kaggle API by running pip install kaggle. To use the Kaggle API, you need to sign up for a Kaggle account at https://www.kaggle.com. Then go to the ‘Account’ tab of your user profile (https://www.kaggle.com/<username>/account) and select ‘Create API Token’. This will trigger the download of kaggle.json, a file containing your API credentials. Place this file in the location ~/.kaggle/kaggle.json on your machine.

In [ ]:
api = KaggleApi()
api.authenticate()
api.dataset_download_files(
    "fedesoriano/heart-failure-prediction",
    path=DATA_DIR,
    unzip=True,
)

In [ ]:
df = pd.read_csv(os.path.join(DATA_DIR, "heart.csv"))
df.head()

In [ ]:
print(df.describe().T)

In [ ]:
fig = px.pie(df, names="Sex")

fig.update_layout(
    title="Sex Distribution",
)

fig.show()

## Adding figures to report
We can add figures and diagrams to report. We can define caption and the section of the report that this figure belongs to. Since we are exploring the distribution of different features in the dataset, we add it to `datasets` section:

In [ ]:
report.log_plotly_figure(
    fig=fig,
    caption="Sex Distribution",
    section_name="datasets",
)

### Age distribution figure
We plot a histogram of ages similarly and add the figure to our report. This will appear under **Dataset** section:

In [ ]:
fig = px.histogram(df, x="Age")
fig.update_layout(
    title="Age Distribution",
    xaxis_title="Age",
    yaxis_title="Count",
    bargap=0.2,
)

fig.show()

### Outcome distribution
Plot outcome distribution and add it to report. Similarly, we also place it under **Dataset** section:

In [ ]:
df["outcome"] = df["HeartDisease"].astype("int")
df = df.drop(columns=["HeartDisease"])

In [ ]:
fig = px.pie(df, names="outcome")
fig.update_traces(textinfo="percent+label")
fig.update_layout(title_text="Outcome Distribution")
fig.update_traces(
    hovertemplate="Outcome: %{label}<br>Count: \
    %{value}<br>Percent: %{percent}",
)
fig.show()

In [ ]:
report.log_plotly_figure(
    fig=fig,
    caption="Outcome Distribution",
    section_name="datasets",
)

In [ ]:
class_counts = df["outcome"].value_counts()
class_ratio = class_counts[0] / class_counts[1]
print(class_ratio, class_counts)

## Data Preprocessing

This dataset does not have any null values, so we can jump to feature scaling. The string data in the dataframe is in the form of object, we need to convert it back to string to work on it:

In [ ]:
string_col = df.select_dtypes(include="object").columns
df[string_col] = df[string_col].astype("string")
string_col = df.select_dtypes("string").columns.to_list()

We are going to apply tree-based models to our data, so we use `LabelEncoder`:

In [ ]:
target = "outcome"

df_processed = pd.get_dummies(df, columns=string_col, drop_first=False)
df_processed.head()

In [ ]:
df.head()

In [ ]:
feature_cols = df_processed.columns.to_list()
feature_cols.remove(target)

Before training, let's document dataset in the model card.
This can be done using the log_dataset method, which takes the following arguments: 

- `description`: A description of the dataset. 
- `citation`: The citation for the dataset. 
- `link`: A link to a resource for the dataset. 
- `license_id`: The SPDX license identifier for the dataset. 
- `version`: The version of the dataset. 
- `features`: A list of features in the dataset. 
- `split`: The split of the dataset (train, test, validation, etc.). 
- `sensitive_features`: A list of sensitive features used to train/evaluate the model. 
- `sensitive_feature_justification`: A justification for the sensitive features used to train/evaluate the model.


In [ ]:
report.log_dataset(
    description="""This dataset was created by combining different datasets
    already available independently but not combined before. In this dataset,
    5 heart datasets are combined over 11 common features. Every dataset used
    can be found under the Index of heart disease datasets from UCI
    Machine Learning Repository on the following link:
    https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/.""",
    citation=inspect.cleandoc(
        """
        @misc{fedesoriano,
          title={Heart Failure Prediction Dataset.},
          author={Fedesoriano, F},
          year={2021},
          publisher={Kaggle}
        }
    """,
    ),
    link="""
    https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction
    """,
    license_id="CC0-1.0",
    version="Version 1",
    features=df.columns.to_list().remove(target),
    sensitive_features=["Sex", "Age"],
    sensitive_feature_justification="Demographic information like age and gender \
        often have a strong correlation with health outcomes. For example, older \
        patients are more likely to have a higher risk of heart disease.",
)

## Create HuggingFace Dataset
We convert our processed Pandas dataframe into a Hugging Face dataset, for later evaluation by Cyclop metrics.

In [ ]:
dataset = Dataset.from_pandas(df_processed)
dataset.cleanup_cache_files()
print(dataset)

In [ ]:
# Split dataframe into inputs and outputs
X, y = df_processed[feature_cols], df_processed[target]

## Training

In [ ]:
# Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=RANDOM_SEED,
)
classifier = LogisticRegression()
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
y_pred_prob = classifier.predict_proba(X_test)

## Evaluation
As demonstrated in evaluation tutorial, we define a metric dict:

In [ ]:
metric_names = [
    "binary_accuracy",
    "binary_precision",
    "binary_recall",
    "binary_f1_score",
    "binary_roc_curve",
    "binary_auroc",
]
metrics = [
    create_metric(metric_name, experimental=True) for metric_name in metric_names
]
metric_collection = MetricDict(metrics)
metric_collection(y_test.values, np.float64(y_pred))

## Data Slicing
In addition to overall metrics, it might be useful to see how the model performs on certain subpopulation or subsets. We can define these subsets using SliceSpec objects.

In [ ]:
spec_list = [
    {
        "Age": {
            "min_value": 30,
            "max_value": 50,
            "min_inclusive": True,
            "max_inclusive": False,
        },
    },
    {
        "Age": {
            "min_value": 50,
            "max_value": 70,
            "min_inclusive": True,
            "max_inclusive": False,
        },
    },
]
slice_spec = SliceSpec(spec_list)

Below, we are combining the raw features of the test data and the predictions so that we use them later for slice-specific evaluations.

In [ ]:
# Get positions of matching indices in df
matching_positions = y_test.index.get_indexer(df.index)

# Select rows from df using matching positions (valid positions are non-negative)
df_test = df.iloc[matching_positions[matching_positions >= 0]]
df_test["preds"] = y_pred
df_test["preds_prob"] = y_pred_prob[:, 1]
df_test.head()

### Age distribution in test data

In [ ]:
fig = px.histogram(df_test, x="Age")
fig.update_layout(
    title="Age Distribution in Test Data",
    xaxis_title="Age",
    yaxis_title="Count",
    bargap=0.2,
)

fig.show()

## Logging metrics and results to report
Here, we gather evalutaions and add them to the report.

We can add a performance metric to the model card using the `log_performance_metric` method, which expects a dictionary where the keys are in the following format: `slice_name/metric_name`. For instance, `overall/accuracy` or `Age:[30 - 50)/BinaryPrecision`.

We first need to process the evaluation results to get the metrics in the right format.

In [ ]:
# Create Dataset object
heart_failure_data = Dataset.from_pandas(df_test)

result = evaluator.evaluate(
    dataset=heart_failure_data,
    metrics=metric_collection,  # type: ignore[list-item]
    target_columns=target,
    prediction_columns="preds_prob",
    slice_spec=slice_spec,
)

In [ ]:
results_flat = flatten_results_dict(
    results=result, remove_metrics=["BinaryROC"], model_name="model_for_preds_prob"
)

We first need to process the evaluation results to get the metrics in the right format. The descriptions dictionary will appear as you hover on metrics in the report, so feel free to change them as it's appropriate for your usage.

In [ ]:
for name, metric in results_flat.items():
    split, name = name.split("/")  # noqa: PLW2901
    descriptions = {
        "BinaryPrecision": "The proportion of predicted positive instances that are correctly predicted.",
        "BinaryRecall": "The proportion of actual positive instances that are correctly predicted. Also known as recall or true positive rate.",
        "BinaryAccuracy": "The proportion of all instances that are correctly predicted.",
        "BinaryAUROC": "The area under the receiver operating characteristic curve (AUROC) is a measure of the performance of a binary classification model.",
        "BinaryAveragePrecision": "The area under the precision-recall curve (AUPRC) is a measure of the performance of a binary classification model.",
        "BinaryF1Score": "The harmonic mean of precision and recall.",
    }
    report.log_quantitative_analysis(
        "performance",
        name=name,
        value=metric.tolist(),
        description=descriptions[name],
        metric_slice=split,
        pass_fail_thresholds=0.7,
        pass_fail_threshold_fns=lambda x, threshold: bool(x >= threshold),
    )

We can also use the `ClassificationPlotter`(as demonstrated in Evaluation example) to plot the performance metrics and the add the figure to the model card using the log_plotly_figure method.

In [ ]:
plotter = ClassificationPlotter(task_type="binary", class_names=["0", "1"])
plotter.set_template("plotly_white")

In [ ]:
# extracting the ROC curves and AUROC results for all the slices
model_name = "model_for_preds_prob"
roc_curves = {
    slice_name: slice_results["BinaryROC"]
    for slice_name, slice_results in result[model_name].items()
}
aurocs = {
    slice_name: slice_results["BinaryAUROC"]
    for slice_name, slice_results in result[model_name].items()
}

# plotting the ROC curves for all the slices
roc_plot = plotter.roc_curve_comparison(roc_curves, aurocs=aurocs)
report.log_plotly_figure(
    fig=roc_plot,
    caption="ROC Curve for All Patients",
    section_name="quantitative analysis",
)
roc_plot.show()

In [ ]:
# Extracting the overall classification metric values.
overall_performance = {
    metric_name: metric_value
    for metric_name, metric_value in result[model_name]["overall"].items()
    if metric_name not in ["BinaryROC", "BinaryPrecisionRecallCurve"]
}
# Plotting the overall classification metric values.
overall_performance_plot = plotter.metrics_value(
    overall_performance,
    title="Overall Performance",
)
report.log_plotly_figure(
    fig=overall_performance_plot,
    caption="Overall Performance",
    section_name="quantitative analysis",
)
overall_performance_plot.show()

In [ ]:
# Extracting the metric values for all the slices.
slice_metrics = {
    slice_name: {
        metric_name: metric_value
        for metric_name, metric_value in slice_results.items()
        if metric_name not in ["BinaryROC", "BinaryPrecisionRecallCurve"]
    }
    for slice_name, slice_results in result[model_name].items()
}
# Plotting the metric values for all the slices.
slice_metrics_plot = plotter.metrics_comparison_bar(slice_metrics)
report.log_plotly_figure(
    fig=slice_metrics_plot,
    caption="Slice Metric Comparison",
    section_name="quantitative analysis",
)
slice_metrics_plot.show()

## Report Generation

### ModelCard
First, let's document the model details section. A **ModelCard** has several **Sections** and each **Section** includes multiple **Fields**. Model details could be one of the sections in our model card, and it has the following fields by default:

- **`description`**: A high-level description of the model and its usage for a general audience. 
- **`version`**: The version of the model. 
- **`owners`**: The individuals or organizations that own the model. 
- **`license`**: The license under which the model is made available. 
- **`citation`**: The citation for the model. 
- **`references`**: Links to resources that are relevant to the model. 
- **`path`**: The path to where the model is stored. 
- **`regulatory_requirements`**: The regulatory requirements that are relevant to the model.

We can add additional fields to the model details section by passing a dictionary to the `log_from_dict` method and specifying the section name as `model_details`. You can also use the `log_descriptor` method to add a new field object with a description attribute to any section of the model card.

In [ ]:
report.log_from_dict(
    data={
        "name": "Heart Failure Prediction Model",
        "description": "The model was trained on the Kaggle Heart Failure \
        Prediction Dataset to predict risk of heart failure.",
    },
    section_name="model_details",
)

report.log_version(
    version_str="0.0.1",
    date=str(date.today()),
    description="Initial Release",
)
report.log_owner(
    name="CyclOps Team",
    contact="vectorinstitute.github.io/cyclops/",
    email="cyclops@vectorinstitute.ai",
)
report.log_license(identifier="Apache-2.0")
report.log_reference(
    link="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html",  # noqa: E501
)

### Considerations

Next, let’s populate the considerations section, which includes the following fields by default: 
- **`users`**: The intended users of the model. 
- **`use_cases`**: The use cases for the model. These could be primary, downstream or out-of-scope use cases. 
- **`fairness_assessment`**: A description of the benefits and harms of the model for different groups as well as the steps taken to mitigate the harms. 
- **`ethical_considerations`**: The risks associated with using the model and the steps taken to mitigate them. This can be populated using the log_risk method.

In [ ]:
report.log_from_dict(
    data={
        "users": [
            {"description": "Hospitals"},
            {"description": "Clinicians"},
        ],
    },
    section_name="considerations",
)
report.log_user(description="ML Engineers")
report.log_use_case(
    description="Predicting risk of heart failure.",
    kind="primary",
)
report.log_use_case(
    description="Predicting risk of pathologies and conditions other\
    than heart failure.",
    kind="out-of-scope",
)
report.log_fairness_assessment(
    affected_group="sex, age",
    benefit="Improved health outcomes for patients.",
    harm="Biased predictions for patients in certain groups (e.g. older patients) \
        may lead to worse health outcomes.",
    mitigation_strategy="We will monitor the performance of the model on these groups \
        and retrain the model if the performance drops below a certain threshold.",
)
report.log_risk(
    risk="The model may be used to make decisions that affect the health of patients.",
    mitigation_strategy="The model should be continuously monitored for performance \
        and retrained if the performance drops below a certain threshold.",
)

### Exporting report
Once the model card is populated, you can generate the report using the `export` method. The report is generated in the form of an HTML file. A JSON file containing the model card data will also be generated along with the HTML file. By default, the files will be saved in a folder named `cyclops_reports` in the current working directory. You can change the path by passing a `output_dir` argument when instantiating the `ModelCardReport` class.

In [ ]:
np.random.seed(42)

synthetic_timestamps = pd.date_range(
    start="1/1/2020", periods=10, freq="D"
).values.astype(str)


report._model_card.overview = None
report_path = report.export(
    output_filename="heart_failure_report_periodic.html",
    synthetic_timestamp=synthetic_timestamps[0],
    last_n_evals=3,
)

shutil.copy(f"{report_path}", ".")
metric_save = None
for i in tqdm(range(len(synthetic_timestamps[1:]))):
    if i == 3:
        report._model_card.quantitative_analysis.performance_metrics.append(
            metric_save,
        )
    report._model_card.overview = None
    for metric in report._model_card.quantitative_analysis.performance_metrics:
        metric.value = np.clip(
            metric.value + np.random.normal(0, 0.1),
            0,
            1,
        )
        metric.tests[0].passed = bool(metric.value >= 0.7)
    if i == 2:
        metrics = []
        for metric in report._model_card.quantitative_analysis.performance_metrics:
            if metric.type == "BinaryAccuracy" and metric.slice == "Age:[30 - 50)":
                metric_save = copy.deepcopy(metric)
            else:
                metrics.append(metric)
        report._model_card.quantitative_analysis.performance_metrics = metrics
    report_path = report.export(
        output_filename="heart_failure_report_periodic.html",
        synthetic_timestamp=synthetic_timestamps[i + 1],
        last_n_evals=3,
    )
    shutil.copy(f"{report_path}", ".")
shutil.rmtree("./cyclops_report")

You're now able to view the report [report](./heart_failure_report_periodic.html).